## Imports

### Explore data

In [1]:
## Imports
from sklearn.neighbors import NearestNeighbors
import numpy as np 
import pandas as pd 

import math
import scipy  

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score

from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import plot## Exploratory Data Analysis

#import warnings
#warnings.filterwarnings('ignore')

## Exploratory Data Analysis

### Import data

In [2]:
datasets=["abalone9-18.csv","Breast.csv","ecoli-0-1_vs_2-3-5.csv","ecoli-0-1_vs_5.csv","ecoli-0-1-4-7_vs_5-6.csv",
"ecoli-0-2-3-4_vs_5.csv","ecoli-0-4-6_vs_5.csv","ecoli-0-6-7_vs_5.csv","ecoli2.csv","ecoli3.csv", "glass0123vs456.csv",
"glass0.csv","glass1.csv","glass6.csv","haberman.csv","iris1.csv","leaf.csv","new-thyroid1.csv","new-thyroid2.csv",
"parkinsons.csv","seeds.csv","spect.csv","wpbc.csv","yeast-1_vs_7.csv","yeast-2_vs_4.csv",
"abalone11–17.csv","abalone4–8.csv","abalone5–10.csv","BreastCancerWisconsin.csv","ecoli1.csv","eligibility-loan.csv",
"iris0.csv","lung-cancer.csv","Maternal-Risk-lmvsh.csv","page-blocks1vs2345.csv","page-blocks2vs4.csv","page-blocks3vs5.csv",
"pima-indians-diabetes.csv","pima.csv","wheat1.csv","wilt.csv",
"winequality-red-3456vs78.csv","winequality-red-34vs56.csv","wisconsin.csv","yeast1.csv","yeast3.csv"]
k=10 #number of epocks

### Data Visualization

In [3]:
def visualise(df):
    fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "bar"}]])
    colors = ['pink', 'skyblue'] 
    fig.add_trace(go.Pie(labels=df[df.columns[-1]].value_counts().index,
                                 values=df[df.columns[-1]].value_counts().values), 1, 1)

    fig.update_traces(hoverinfo='label+percent', textfont_size=20,
                      marker=dict(colors=colors, line=dict(color='#000000', width=2)))

    fig.add_trace(go.Bar(x=df[df.columns[-1]].value_counts().index, y=df[df.columns[-1]].value_counts().values, marker_color = colors), 1,2)

    fig.show()

## Model developement

In [4]:
def get_labels(df):
    valuecounts=df[df.columns[-1]].value_counts().index
    majlabel=valuecounts[0]
    minlabel=valuecounts[1:]
    if len(minlabel)==1:
        minlabel=int(minlabel[0])
    return minlabel,int(majlabel)

In [5]:
def findNeighbours(maj,df):
    #find neighbours for each instance in the whole dataset 
    neigh = NearestNeighbors(n_neighbors=6)
    neigh.fit(df)
   # NearestNeighbors(n_neighbors=5) change it to six 
    NNAllDatasetar = neigh.kneighbors(df, return_distance=False)
    NNAllDataset = pd.DataFrame(NNAllDatasetar, columns = ['index','n1','n2','n3','n4','n5'])
    
    #find the subset of the instances that belong to the majority class
    NNMAJ = NNAllDataset[NNAllDataset.index.isin(maj.index)]
    NNMAJ = NNMAJ.drop(['index'], axis=1)
    NNMAJ =pd.merge(maj, NNMAJ, left_index=True, right_index=True)
    return(NNMAJ)
    

In [6]:
def calculateweight(df,maj):
    #  add  a column in which for each neighbooring point by 1 if it belongs to the majority class and 0 if it belongs to the minority class
    df.insert(len(df.columns) ,"n1W", np.where(df["n1"].isin(maj.index) , 1, 0))
    df.insert(len(df.columns) ,"n2W", np.where(df["n2"].isin(maj.index) , 1, 0))
    df.insert(len(df.columns) ,"n3W", np.where(df["n3"].isin(maj.index) , 1, 0))
    df.insert(len(df.columns) ,"n4W", np.where(df["n4"].isin(maj.index) , 1, 0))
    df.insert(len(df.columns) ,"n5W", np.where(df["n5"].isin(maj.index) , 1, 0))
    df
    
    #  Calculate the weight 
    df.insert( len(df.columns) ,"weight", df[['n1W', 'n2W','n3W', 'n4W', 'n5W']].mean(axis=1),True)
    
    #  Sort the dataframe by the weight 
    df=df.sort_values(by=['weight'] ,ascending=False)
    
    return df

In [7]:
#def repPoints(maj,df,n):
 #   resultfinal= df.head(n)
  #  return maj[maj.index.isin(resultfinal.index)]

In [8]:

def calculate(df,minlabel, majlabel):
    """
    
    """
    P = (df[df.columns[-1]] == minlabel).sum() #Number of instances in the minority class
    N = (df[df.columns[-1]] == majlabel).sum() #Number of instances in the majority class
   
    n1 = P**2/N
    
    majclass = df[df[df.columns[-1]] == majlabel]
    sigma = np.var(majclass.to_numpy()) #Strandard deviation of the majority class  # square root variance formula
    Zalpha = scipy.stats.norm.ppf(0.5) #the critical value of the Z test at the significance level α
    epsilone =  pow(10,-4) # acceptable tolerance error that can be adjusted as required 10 power -4
    
    n2= (N*(Zalpha**2)*sigma)/((N*epsilone**2)+((Zalpha**2)*sigma))
    
    pr = n2/n1
    
    M = 1.5
    
    if pr < 1:
         size = n1
    elif pr > M:
         size = n1*M
    else:
         size = n2
    return size
    

In [9]:
def concatinate(df,ds):
    ds1 = df 
    ds1 = ds1.iloc[: , :-1]
    ds2 = ds
   # ds1 = ds1.drop(ds1.index[0]) #if the same instance can't be concatinated with itself 
    
    mainds= pd.DataFrame()
    m=len(ds2.index)
    for p in range(m):
        mainds = mainds.append((ds1.assign(key=1).merge(ds2.head(1).assign(key=1), on='key').drop('key',axis=1)), ignore_index=True)
        ds2 = ds2.iloc[1: , :] 
    return (mainds)

In [10]:
#figure out the subset
def representeticepointssel(df,c):
    if df.size<=c: #number of rows
        tmpSet = df
    else:
        tmpSet = []
        vecB= np.mean(df.to_numpy())
        for i in range(c):
            maxDist = 0
            for p in df.iterrows():  # iterate it by rows not columns
                if i==0:
                    minDist = scipy.spatial.distance.pdist((p, vecB),'euclidean') #vecA is p vecB mean value of all the rows in df
                else:
                    # for a given p, if p's min distance to any q in tmpset is biggest, then p is next representative point 
                    minDist = np.min([scipy.spatial.distance.pdist((p, q),'euclidean') for q in tmpSet])
                if minDist >= maxDist:
                    maxPoint = p
                    maxDist = minDist
            tmpSet.append(maxPoint)
        return(tmpSet)

In [11]:
def concattest(df,finalMaj):
    ds1 = df.iloc[: , :-1]
    df = pd.concat([ds1, df], axis=1)
    df.columns = list(finalMaj.columns)
    return df

In [12]:
def repPoints(neighboursds,df,n):

    rep1 = neighboursds[neighboursds["weight"] > neighboursds.head(n)["weight"].iloc[-1]]
    rep2 = neighboursds[neighboursds["weight"] == neighboursds.head(n)["weight"].iloc[-1]]

    rep1 =  df[df.index.isin(rep1.index)]
    rep2 =  df[df.index.isin(rep2.index)]

    if  (neighboursds.head(n)["weight"].iloc[-1] == neighboursds.head(n+1)["weight"].iloc[-1]):
        if (len(rep1.index)>0):
            m=n-len(rep1.index)
            rep2= df.head(m) #representeticepointssel(rep2,m)
            resultfinal= rep1.append(rep2)
        else :
            rep2= df.head(n) #representeticepointssel(rep2,n)
            resultfinal= rep1.append(rep2)   

    else :
        resultfinal= df.head(n) 
    return resultfinal


In [13]:
def con(dftrain,dftest,df):
    minlabel, majlabel= get_labels(dftrain)
    majds = dftrain[dftrain[dftrain.columns[-1]] == majlabel]
    minds = dftrain[dftrain[dftrain.columns[-1]] == minlabel]
    

    IRO= len(majds.index)/len(minds.index)

    neighboursds = findNeighbours(majds,df)
    neighboursds = calculateweight(neighboursds,majds)

    size = calculate(dftrain,minlabel, majlabel)

    reppoints=repPoints(neighboursds,majds,math.ceil(size)) # use math.ceil(size)  or math math.floor(size)


    finalMaj=concatinate(majds,reppoints)
    finalMin=concatinate(minds,minds) 

    IRcon= len(finalMaj.index)/len(finalMin.index)

    finaldf=pd.concat([finalMaj,finalMin])
    finaltest =concattest(dftest,finalMaj)   

    #compare.loc[len(compare.index)] = [dataset, len(dftrain.index), len(majds.index),len(minds.index),IRO,len(reppoints.index),len(finaldf.index),len(finalMaj.index),len(finalMin.index),IRcon] 
    return finaldf, finaltest

In [14]:
def scores(y_test,predictions):
    AUC=0
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
    accuracy= round(balanced_accuracy_score(y_test, predictions)*100, 2)
    F1 = round(f1_score(y_test, predictions, average='weighted')*100, 2)
    precision = round(precision_score(y_test, predictions, average='weighted')*100, 2)
    recall = round(recall_score(y_test, predictions, average='weighted')*100, 2)
    try:
        AUC = round(roc_auc_score(y_test, predictions, average='weighted')*100, 2)
    except ValueError:
        pass
    return accuracy,F1,precision,recall,AUC,tn, fp, fn, tp

In [15]:
from sklearn.metrics import confusion_matrix
def epochsScore(dataset,k):

    path="splitdatasets/"+dataset+"/"+dataset
    compare = pd.DataFrame(columns=['iteration','accuracy','F1','precision',"recall","AUC","tn", "fp", "fn", "tp"])

    for x in range(k):
        train=pd.read_csv(path+'_train_'+str(x)+'.csv',sep='\t', index_col=0)
        test=pd.read_csv(path+'_test_'+str(x)+'.csv',sep='\t', index_col=0)
        df = pd.read_csv("datasets/"+dataset)
        train,test= con(train,test,df)
        
        lda_con = RandomForestClassifier()       
        X_train_con = train.drop(train.columns[-1], axis=1)
        X_test_con = test.drop(test.columns[-1], axis=1)
        

        y_train_con = train[train.columns[-1]]
        y_test_con = test[test.columns[-1]]
        
        lda_con.fit(X_train_con, y_train_con)
        predictions_con = lda_con.predict(X_test_con)
        accuracy,F1,precision,recall,AUC,tn, fp, fn, tp =  scores(y_test_con,predictions_con)
        compare.loc[len(compare.index)]=[int(x), accuracy,F1,precision,recall,AUC,tn, fp, fn, tp]
    avacc = compare["accuracy"].mean()
    avF1 = compare["F1"].mean()
    avPrecision = compare["precision"].mean()
    avrecall = compare["recall"].mean()
    avAUC = compare["AUC"].mean()
    avtn = compare["tn"].mean()
    avfp = compare["fp"].mean()
    avfn = compare["fn"].mean()
    avtp = compare["tp"].mean()
    
    compare.loc[len(compare.index)]=["average", avacc,avF1,avPrecision,avrecall,avAUC,avtn,avfp,avfn,avtp]

    return avacc,avF1,avPrecision,avrecall,avAUC,avtn,avfp,avfn,avtp

In [16]:
def complda(datasets,k):
    classificationCompare = pd.DataFrame(columns=['Dataset','Version','Accuracy score %', "F1 Score %",'precision score %', "recall Score %", "AUC Score %","tn", "fp", "fn", "tp"])

    for dataset in datasets:
        #classification on original dataset
        df = pd.read_csv("datasets/"+dataset)
        print(dataset)
        X = df.drop(df.columns[-1], axis=1)
        y = df[df.columns[-1]]
        
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=44)
        lda_or = RandomForestClassifier()
        lda_or.fit(X_train, y_train)
        
        predictions_or = lda_or.predict(X_test)
        
        accuracy,F1,precision,recall,AUC,tn, fp, fn, tp =  scores(y_test,predictions_or)
        classificationCompare.loc[len(classificationCompare.index)]=[dataset,"original", accuracy,F1,precision,recall,AUC,tn, fp, fn, tp]
        #classification on concatinated dataset
        
        accuracy,F1,precision,recall,AUC,tn, fp, fn, tp =  epochsScore(dataset,k)
        classificationCompare.loc[len(classificationCompare.index)]=[dataset,"concatinated", accuracy,F1,precision,recall,AUC,tn, fp, fn, tp]

    return classificationCompare 

In [17]:
classificationCompare = complda(datasets,k)

abalone9-18.csv


C:\Users\donia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Breast.csv
ecoli-0-1_vs_2-3-5.csv


C:\Users\donia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



ecoli-0-1_vs_5.csv
ecoli-0-1-4-7_vs_5-6.csv
ecoli-0-2-3-4_vs_5.csv
ecoli-0-4-6_vs_5.csv
ecoli-0-6-7_vs_5.csv
ecoli2.csv
ecoli3.csv
glass0123vs456.csv
glass0.csv
glass1.csv
glass6.csv
haberman.csv
iris1.csv
leaf.csv
new-thyroid1.csv
new-thyroid2.csv
parkinsons.csv
seeds.csv
spect.csv
wpbc.csv
yeast-1_vs_7.csv
yeast-2_vs_4.csv
abalone11–17.csv


C:\Users\donia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



abalone4–8.csv
abalone5–10.csv
BreastCancerWisconsin.csv
ecoli1.csv
eligibility-loan.csv
iris0.csv
lung-cancer.csv
Maternal-Risk-lmvsh.csv
page-blocks1vs2345.csv
page-blocks2vs4.csv
page-blocks3vs5.csv
pima-indians-diabetes.csv
pima.csv
wheat1.csv
wilt.csv
winequality-red-3456vs78.csv
winequality-red-34vs56.csv


C:\Users\donia\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



wisconsin.csv
yeast1.csv
yeast3.csv


In [18]:
classificationCompare.head(50)

,Dataset,Version,Accuracy score %,F1 Score %,precision score %,recall Score %,AUC Score %,tn,fp,fn,tp
0,abalone9-18.csv,original,50.000,95.960,94.670,97.300,50.000,72,0,2,0
1,abalone9-18.csv,concatinated,82.130,72.145,95.696,66.298,82.130,44.3,24.6,0.0,4.2
2,Breast.csv,original,90.950,92.980,92.980,92.980,90.950,13,2,2,40
3,Breast.csv,concatinated,100.000,100.000,100.000,100.000,100.000,21.2,0.0,0.0,35.7
4,ecoli-0-1_vs_2-3-5.csv,original,50.000,94.040,92.160,96.000,50.000,24,0,1,0
5,ecoli-0-1_vs_2-3-5.csv,concatinated,98.636,97.953,99.027,97.500,98.636,21.4,0.6,0.0,2.4
6,ecoli-0-1_vs_5.csv,original,97.730,96.200,97.220,95.830,97.730,21,1,0,2
7,ecoli-0-1_vs_5.csv,concatinated,98.408,97.544,98.666,97.084,98.408,21.3,0.7,0.0,2.0
8,ecoli-0-1-4-7_vs_5-6.csv,original,85.710,93.730,94.520,94.120,85.710,27,0,2,5
9,ecoli-0-1-4-7_vs_5-6.csv,concatinated,95.343,92.537,98.657,91.287,95.343,27.9,2.8,0.0,2.5


In [19]:
def complda(original,concatinated , x, classificationCompare):
    for number in range(0, len(classificationCompare.index) , 2):
        if(  classificationCompare.iat[number,x] > classificationCompare.iat[number+1,x]):
            original=original+1
        else:
            concatinated=concatinated+1
            
    return original, concatinated
     

In [20]:
def compdf(compare, Measure, x , classificationCompare):
    original = 0
    concatinated =0
    original, concatinated = complda(original,concatinated , x , classificationCompare)
    compare.loc[len(compare.index)]=[Measure,original, concatinated]
    return compare


In [21]:
compare = pd.DataFrame(columns=['Measure','Original','Concatinated'])
compdf(compare, "Accuracy", 2 , classificationCompare)
compdf(compare, "F1", 3 , classificationCompare)
compdf(compare, "Precision", 4 , classificationCompare)
compdf(compare, "Recall", 5 , classificationCompare)
compdf(compare, "AUC", 6 , classificationCompare)
compdf(compare, "tn", 7 , classificationCompare)
compdf(compare, "fp", 8 , classificationCompare)
compdf(compare, "fn", 9 , classificationCompare)
compdf(compare, "tp", 10 , classificationCompare)

,Measure,Original,Concatinated
0,Accuracy,10,36
1,F1,16,30
2,Precision,11,35
3,Recall,17,29
4,AUC,10,36
5,tn,20,26
6,fp,24,22
7,fn,21,25
8,tp,15,31
